# 数据治理的本体实现
## dcat本体的载入
实现将dcat本体导入grapdb数据库中，进一步实现通过python读取graphdb。

---

其中，sparql_endpoint的地址为 'http://localhost:7200/repositories/dcat'
其中repositories/dcat 为repositories，数据库名称

如果使用graphdb作为数据存储，好处是可以利用其推理，并可以处理较大规模数据，方便管理。

同时目前在构建rdf语义库时，tbxo与abox没有比较好的分离，以及各个本体混在一起，不方便管理，可以尝试通过graphdb进行管理。

---

结合之前python构建本体方式，实现dcat相关的设计，并直接写入graphdb数据库中

---
> [python导入graphdb](https://stackoverflow.com/questions/70991665/python-library-for-graphdb)

In [7]:
from SPARQLWrapper import SPARQLWrapper, JSON,XML
import sys

# GraphDB SPARQL端点的URL
sparql_endpoint = 'http://localhost:7200/repositories/environment'

# 创建一个SPARQLWrapper对象
sparql = SPARQLWrapper(sparql_endpoint)

# 设置你的SPARQL查询
sparql.setQuery("""
select * where {
    ?s ?p ?o .
} limit 100
""")

# 设置返回格式
sparql.setReturnFormat(JSON)

try:
    # 执行查询并获取结果
    results = sparql.query().convert()

    # 打印整个结果以检查结构
    print(results)

    # 打印结果
    for result in results["results"]["bindings"]:
        print(result["s"]["value"], result["p"]["value"], result["o"]["value"])
except Exception as e:
    print("An error occurred:", e)
    sys.exit(1)

{'head': {'vars': ['s', 'p', 'o']}, 'results': {'bindings': [{'s': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'}}, {'s': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#subPropertyOf'}, 'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'}}, {'s': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#subPropertyOf'}, 'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#TransitiveProperty'}}, {'s': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#subClassOf'}, 'p': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 'o': {'type': '

# 基于named grah的 tbox，abox分离存储

reference：

[named grpah](https://www.w3.org/2009/07/NamedGraph.html)

## TBox and ABox

- **TBox**: Contains terminologies, i.e., the definitions of classes and the relationships between them. It describes the structure of the domain in terms of concepts and their interrelations.
- **ABox**: Contains assertions about instances of those classes defined in the TBox, describing specific instances of classes and their relationships.

## Storing in Separate Named Graphs

You can store the TBox and ABox in separate named graphs within an RDF store:

- **TBox Named Graph**: Might contain triples that define and describe the schema or ontology (e.g., class hierarchies, property definitions).
- **ABox Named Graph**: Contains the actual data or instances that conform to the schema defined in the TBox.

## Pros of Separating TBox and ABox in Named Graphs

1. **Modularity**: Keeping the TBox and ABox in separate graphs enhances modularity. You can update or modify the ontology (TBox) without affecting the instance data (ABox), and vice versa.
2. **Manageability**: It simplifies management and versioning of the ontology. Changes in the conceptual schema can be tracked independently of changes in the data.
3. **Scalability**: Queries can be optimized by targeting specific graphs. For instance, if a query only needs to understand class definitions, it can exclusively query the TBox, thus reducing the search space.
4. **Security and Access Control**: Different access controls can be applied to the TBox and ABox. Typically, more users might have read access to the ABox, while the TBox might be restricted to those who manage and define ontologies.
5. **Performance Optimization**: In large datasets, separating static schema (TBox) from dynamic and potentially large instance data (ABox) can improve performance in loading and querying data.

## Cons of Separating TBox and ABox in Named Graphs

1. **Complexity in Querying**: Queries that need to span both TBox and ABox require more complex SPARQL queries, potentially joining data from multiple graphs, which can lead to performance overhead.
2. **Maintenance Overhead**: Maintaining consistency between the TBox and ABox across separate graphs can be challenging, especially if the schema changes require corresponding updates in the instance data.
3. **Potential for Inconsistencies**: If not carefully managed, there might be risks of schema-instance inconsistencies, where the ABox does not properly reflect the structures defined in the TBox.
4. **Dependency Management**: Changes in the TBox (like adding or modifying classes and properties) need careful management to ensure that the ABox remains consistent with these changes, which can be cumbersome and error-prone.

## Conclusion

Storing the TBox and ABox in separate named graphs can provide significant benefits in terms of modularity, scalability, and manageability, particularly for large and complex datasets or ontologies. However, this approach requires careful implementation to handle the additional complexity in query processing and data management to avoid inconsistencies and ensure performance efficiency.


基于以上优缺点，实验通过不同的named graph 存储知识图谱中的数据，便于分离语义与实例

为了简便，通过web界面进行操作

1. 定义tbox 的named graph url=from http://tom.whu/dcat/tbox
2. 通过web界面直接导入rdf文件到named graph中
3. 实现查询
4. 之后的对tbox的操作，就应当面向该named graph

```sql 
select * from <http://tom.whu/dcat/tbox>

where {
    ?s ?p ?o .
} limit 100
```
基于named graph，为了实现数据治理，首先导入相关的本体，包括

* dcat 
* prov 
* odrl 
* skos
-------
同时abox来源于环保厅提供的相关文件，其文件包含以下几个内容模块

* 贵州省生态环境大数据中心建设项目安全保障体系规范
> 对应的本体包括odrl，skos

* 贵州省生态环境大数据中心建设项目统一用户标准规范
> 对应的本体包括org，foaf

*贵州省生态环境大数据中心建设项目数据交换标准规范

首先从较为简单的skos入手，建立保障体系规范中的conception

----
设计对应的skos时，应该考虑到几点
1. 首先对应的skos来源于一个dataset（文档），那么改文档本身就应该作为一种资源，由dcat进行标识描述

2. 对应该dataset,应该基于dc进行描述，而原始文档里面有关于作者，版权，日期等信息，可以对应

3. 
基于skos的定义与组织，首先我们应该定义一个概念体系（

以及定义一个collection以容纳对应的概念

基本的概念模型包括
``` json
zz:c1 rdf:type skos:Conception;
skos:definition xxxxxx @cn;

```

In [ ]:
!pip install pip install graphdb-python
!pip install python-docx

In [8]:
 
from docx import Document
concpt_dict={}
def read_docx_headings(docx_file):
    doc = Document(docx_file)
    is_heading = False
    for paragraph in doc.paragraphs:
        if paragraph.style.name == 'Heading 3':
            if is_heading:  # 如果已经读取过标题，则打印标题和相应的内容
                print(f"内容: {text}")
            heading_text = paragraph.text
            print(f"标题 3: {heading_text}")
            is_heading = True
            text = ""
        else:
            if is_heading:  # 如果当前段落不是标题，但是上一个段落是标题，则将其内容添加到text变量中
                text += paragraph.text

    # 处理最后一个标题
    if is_heading:
        print(f"内容: {text}")

# 替换为你的Word文档路径
docx_file_path = r"C:\data\KG\terminology.docx"
read_docx_headings(docx_file_path)

标题 3: 3.1.1 事故 accident
内容: 造成人员伤亡,职业疾病,设备、财产损坏或损失,环境破坏等的一个或一系列非预期事件。
标题 3: 3.1.2 组件 component
内容: 一个系统或子系统的组成元素。
标题 3: 3.1.3 客户 customer
内容: 获取其他组织所开发的软件的实体,包括工程、项目、设施。
标题 3: 3.1.4 分解 decomposition
内容: 将一个系统或组件划分为组成部分的过程。
标题 3: 3.1.5 失效模式与影响	的分析 failure modes and effects analysis
内容: 一种自底向上系统的、归纳的、有条理的分析,用于在指定的级别上标识并记录所有可识别的失效   模式并详述失效模式的后果。
标题 3: 3.1.6 故障树分析 fault tree analysis
内容: 一种故障分析技术,用来识别非期望的系统状态,在系统环境和运行的上下文中对其进行分析以找到所有非期望事件可能出现的可信途经。
标题 3: 故障检测 fault detection
内容: 发现故障的能力，判断故障已经发生的过程。
标题 3: 3.1.8 故障隔离 fault isolation
内容: 确定故障位置或源头的过程。
标题 3: 3.1.9 故障恢复 fault recovery
内容: 不必经过长时间的重配置,系统就可以克服故障的过程。
标题 3: 3.1.10 功能需求 functional requirements
内容: 功能需求定义了系统或子系统为完成它的任务应做什么,以及时间及性能需求。
标题 3: 3.1.11 危害 hazard
内容: 可能导致或引发灾祸或意外事故的现存或潜在的情况。
标题 3: 3.1.12 危害控制 hazard control
内容: 减少因危害发生而导致的风险的手段,包括用于减小危害后果发生的可能性、降低危害严重程度的设计或运行特性。
标题 3: 3.1.13 危害缓解 hazard mitigation
内容: 减少或消除由危害导致的风险的任何措施。
标题 3: 3.1.14 协定备忘录 memorandum of agreement
内容: 双方或多方之间的书面协定,它定义了与某一特定方案或项目协同工作有关各方的角色和责任。注:MOA有时也称为谅解备

In [9]:
from docx import Document

def read_docx_headings(docx_file):
    doc = Document(docx_file)
    is_heading = False
    data = {}
    for paragraph in doc.paragraphs:
        if paragraph.style.name == 'Heading 3':
            if is_heading:
                data[heading_text] = text  # 将标题和内容添加到字典中
            heading_text = paragraph.text
            is_heading = True
            text = ""
        else:
            if is_heading:
                text += paragraph.text

    # 处理最后一个标题
    if is_heading:
        data[heading_text] = text

    return data

# 替换为你的Word文档路径
docx_file_path = r"C:\data\KG\terminology.docx"
result_dict = read_docx_headings(docx_file_path)
print(result_dict)

{'3.1.1 事故 accident': '造成人员伤亡,职业疾病,设备、财产损坏或损失,环境破坏等的一个或一系列非预期事件。', '3.1.2 组件 component': '一个系统或子系统的组成元素。', '3.1.3 客户 customer': '获取其他组织所开发的软件的实体,包括工程、项目、设施。', '3.1.4 分解 decomposition': '将一个系统或组件划分为组成部分的过程。', '3.1.5 失效模式与影响\t的分析 failure modes and effects analysis': '一种自底向上系统的、归纳的、有条理的分析,用于在指定的级别上标识并记录所有可识别的失效   模式并详述失效模式的后果。', '3.1.6 故障树分析 fault tree analysis': '一种故障分析技术,用来识别非期望的系统状态,在系统环境和运行的上下文中对其进行分析以找到所有非期望事件可能出现的可信途经。', '故障检测 fault detection': '发现故障的能力，判断故障已经发生的过程。', '3.1.8 故障隔离 fault isolation': '确定故障位置或源头的过程。', '3.1.9 故障恢复 fault recovery': '不必经过长时间的重配置,系统就可以克服故障的过程。', '3.1.10 功能需求 functional requirements': '功能需求定义了系统或子系统为完成它的任务应做什么,以及时间及性能需求。', '3.1.11 危害 hazard': '可能导致或引发灾祸或意外事故的现存或潜在的情况。', '3.1.12 危害控制 hazard control': '减少因危害发生而导致的风险的手段,包括用于减小危害后果发生的可能性、降低危害严重程度的设计或运行特性。', '3.1.13 危害缓解 hazard mitigation': '减少或消除由危害导致的风险的任何措施。', '3.1.14 协定备忘录 memorandum of agreement': '双方或多方之间的书面协定,它定义了与某一特定方案或项目协同工作有关各方的角色和责任。注:MOA有时也称为谅解备忘录(MOU)。', '3.1.15 任务关键 mission critical': '任务的项或功能应保持运行

In [10]:
import pandas as pd
df = pd.DataFrame(pd.Series(result_dict), columns=['definition'])
df = df.reset_index().rename(columns={'index':'concept'})
# 去除编号
df['concept'] = df['concept'].str.replace(r'^\d+\.\d+\.\d+\s', '', regex=True)

# 分离中文和英文
df['concept_cn'] = df['concept'].str.extract(r'([\u4e00-\u9fff]+)')
df['concept_en'] = df['concept'].str.extract(r'([a-zA-Z ]+)')

# 结果DataFrame
result = df[['concept_cn', 'concept_en', 'definition']]
result

,concept_cn,concept_en,definition
0,事故,accident,"造成人员伤亡,职业疾病,设备、财产损坏或损失,环境破坏等的一个或一系列非预期事件。"
1,组件,component,一个系统或子系统的组成元素。
2,客户,customer,"获取其他组织所开发的软件的实体,包括工程、项目、设施。"
3,分解,decomposition,将一个系统或组件划分为组成部分的过程。
4,失效模式与影响,failure modes and effects analysis,"一种自底向上系统的、归纳的、有条理的分析,用于在指定的级别上标识并记录所有可识别的失效 ..."
5,故障树分析,fault tree analysis,"一种故障分析技术,用来识别非期望的系统状态,在系统环境和运行的上下文中对其进行分析以找到所有..."
6,故障检测,fault detection,发现故障的能力，判断故障已经发生的过程。
7,故障隔离,fault isolation,确定故障位置或源头的过程。
8,故障恢复,fault recovery,"不必经过长时间的重配置,系统就可以克服故障的过程。"
9,功能需求,functional requirements,"功能需求定义了系统或子系统为完成它的任务应做什么,以及时间及性能需求。"


In [11]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, SKOS

# 创建一个图
g = Graph()

# 定义命名空间
ZZ_CON = Namespace("http://tom.whu/conception/")
g.bind("zz_con", ZZ_CON)
g.bind("skos", SKOS)

# 创建一个SKOS Collection
collection_uri = ZZ_CON['大数据中心安全保障体系规范']
g.add((collection_uri, RDF.type, SKOS.Collection))
g.add((collection_uri, SKOS.prefLabel, Literal("贵州省生态环境大数据中心建设项目安全保障体系规范", lang="zh")))

# 假设df是您已经处理好的DataFrame
for index, row in df.iterrows():
    concept_uri = ZZ_CON['concept_' + str(index)]  # 每个概念的唯一URI
    g.add((concept_uri, RDF.type, SKOS.Concept))
    g.add((concept_uri, SKOS.prefLabel, Literal(row['concept_cn'], lang="zh")))
    g.add((concept_uri, SKOS.prefLabel, Literal(row['concept_en'], lang="en")))
    g.add((concept_uri, SKOS.definition, Literal(row['definition'], lang="zh")))
    g.add((collection_uri, SKOS.member, concept_uri))  # 将概念添加为集合的成员

# 保存图到文件
print(g.serialize(format="turtle"))

@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix zz_con: <http://tom.whu/conception/> .

zz_con:大数据中心安全保障体系规范 a skos:Collection ;
    skos:member zz_con:concept_0,
        zz_con:concept_1,
        zz_con:concept_10,
        zz_con:concept_11,
        zz_con:concept_12,
        zz_con:concept_13,
        zz_con:concept_14,
        zz_con:concept_15,
        zz_con:concept_16,
        zz_con:concept_17,
        zz_con:concept_18,
        zz_con:concept_19,
        zz_con:concept_2,
        zz_con:concept_20,
        zz_con:concept_21,
        zz_con:concept_22,
        zz_con:concept_23,
        zz_con:concept_24,
        zz_con:concept_25,
        zz_con:concept_26,
        zz_con:concept_27,
        zz_con:concept_28,
        zz_con:concept_29,
        zz_con:concept_3,
        zz_con:concept_30,
        zz_con:concept_31,
        zz_con:concept_32,
        zz_con:concept_33,
        zz_con:concept_34,
        zz_con:concept_35,
        zz_con:concept_36,
        zz_con:

In [22]:
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, SKOS

# Setup the SPARQL connection
sparql = SPARQLWrapper('http://localhost:7200/repositories/environment')
sparql.setHTTPAuth(DIGEST)
sparql.setMethod(POST)
sparql.setCredentials('admin', 'tomis1cat')

# Serialize the graph to Turtle format
data = g.serialize(format="turtle")

# SPARQL update query for inserting data into a specific named graph
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>

INSERT DATA {
  GRAPH <http://tom.whu/dcat/abox> {
zz_con:concept_8 a skos:Concept ;
    skos:definition "不必经过长时间的重配置,系统就可以克服故障的过程。"@zh ;
    skos:prefLabel " fault recovery"@en,
        "故障恢复"@zh .
  }
}
""" 
 
# Set the query
sparql.setQuery(query)

# Execute the update and handle potential errors
rs=sparql.query()
print(rs)

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Missing parameter: query'

In [54]:
!pip install  pygraphdb

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pygraphdb: filename=pygraphdb-2.0.1-py3-none-any.whl size=12166 sha256=24176aa25d9078d248be612110740549ec202a863d15b55509732b1d607918fb
  Stored in directory: c:\users\tom\appdata\local\pip\cache\wheels\1c\58\b1\15ff4efc49b4facbae37ef48306b7967fa0e8f2ad345dee2d1
Successfully built pygraphdb


https://blog.csdn.net/weixin_44836662/article/details/121525898

使用sqlwrapper更新graphdb数据遇到问题，使用pygraphdb更新错误较少，同时相对功能更多

故此，对graph更新使用pygraphdb库，查询等可以考虑sqlwrapper，以

In [26]:
print(data)

@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix zz_con: <http://tom.whu/conception/> .

zz_con:大数据中心安全保障体系规范 a skos:Collection ;
    skos:member zz_con:concept_0,
        zz_con:concept_1,
        zz_con:concept_10,
        zz_con:concept_11,
        zz_con:concept_12,
        zz_con:concept_13,
        zz_con:concept_14,
        zz_con:concept_15,
        zz_con:concept_16,
        zz_con:concept_17,
        zz_con:concept_18,
        zz_con:concept_19,
        zz_con:concept_2,
        zz_con:concept_20,
        zz_con:concept_21,
        zz_con:concept_22,
        zz_con:concept_23,
        zz_con:concept_24,
        zz_con:concept_25,
        zz_con:concept_26,
        zz_con:concept_27,
        zz_con:concept_28,
        zz_con:concept_29,
        zz_con:concept_3,
        zz_con:concept_30,
        zz_con:concept_31,
        zz_con:concept_32,
        zz_con:concept_33,
        zz_con:concept_34,
        zz_con:concept_35,
        zz_con:concept_36,
        zz_con:

In [25]:
import pygraphdb
data = g.serialize(format="turtle")
# 数据库 config
host = '192.168.31.172'
port = '7200'
db_name = 'environment'
user = 'admin'
password = 'tomis1cat'
sparql = """
prefix zz_con: <http://tom.whu/conception/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/>
INSERT DATA {
  GRAPH <http://tom.whu/dcat/abox> {
zz_con:concept_8 a skos:Concept ;
    skos:definition "不必经过长时间的重配置,系统就可以克服故障的过程。"@zh ;
    skos:prefLabel " fault recovery"@en,
        "故障恢复"@zh .
  }
}
""" 
# 连接数据库
db = pygraphdb.connect(host=host, port=port, user=user, password=password, db=db_name)
# 获取 cursor
cur = db.cursor()
# 执行sparql语句，得到结果
result = cur.execute(sparql)
print(result)
# 关闭 cursor
cur.close()
# 关闭数据库
db.close()
 